# Spatial prediction of soil pollutants with multi-output Gaussian processes

<h2 class="pm-node nj-subtitle">A tutorial with <em>PyMC</em></h2>

[](https://nextjournal.com/data/?node-id=8371e367-647e-4b6b-add0-84d5d74f01a6&node-kind=file)

*Purple sunset, photo of the Meuse river by [Ilirjan Rrumbullaku](https://www.flickr.com/photos/125701341@N07/17007903751/in/photolist-rUVXeR-2mDm2ve-2mCViwV-2mD7Ft6-2mCZC8o-2mmkekw-r6XLHT-2jVTiLX-DumJH8-4vjFYu-SpDSBm-4vfCYD-2eMX17g-gvFG4Q-F1wuSp-nSdjG8-GrgTj6-QTtVNQ-zN7VjX-bR7V2z-2mBsjRu-ewMGd-GwTBDm-5WcGgQ-4vfDhM-YQpN4q-DrYQ3n-muTtNi-2kLrrQi-29diro-Wxukw-SRrbcx-2mmkeFw-2mo1gx9-2mD4ndH-2mo5a7u-2iVkfXk-2mmp7ex-2mD3W8P-2mmhWhZ-2mBwG8U-2mmrDum-2mmmMf1-Bvi2LB-2mD9HbR-4wmpZU-2dbRjY5-2mmhYeK-Ei7Z7B-2iVmQBJ)*

The [Meuse river](https://www.openstreetmap.org/#map=12/50.9619/5.7327) flows through Germany, the Netherlands, Belgium, and France. Several industries have settled on its banks, so that its bed, as well as the soils of its banks, are polluted by metals. A particular area north of Maastricht, the Netherlands, is often used as a case study for geostatistics.

**Knowledge requirements**. Intermediate levels in Python programming and data science. Beginner levels in positionning, Gausian processes and *PyMC*. To reach such level in *PyMC* and Gaussian processes, I suggest [this lecture](https://www.youtube.com/watch?v=j7Ruu3Yu-70) from Chris Fonnesbeck. For positionning, you will get the very basic concepts on [this Vox 6 minute video](https://www.youtube.com/watch?v=kIID5FDi2JQ).

**Specific objectives**. At the end of this tutorial, you will be able to

1. show geographic data on a map background,
2. perform spatial queries,
3. model spatial attributes with Gaussian processes,
4. project spatial attributes on a background map.

**Tools**. We will use Python as programming language with, as packages, *Numpy* (generic mathematical operations), *Pandas* (to handle tabular data), *GeoPandas* (to create spatial data types in tabular data, and perform spatial queries on them), *Lets-Plot* (for interactive plots) and *PyMC* (to build our Gaussian process model).

# Multi-output Gaussian processes

You might need a multi-output prediction when you suppose that the outputs are related to each other (i.e. correlated). Such cases are common is spatial predictions. Without committed ourselves on causality, we can often assume that things at the same location might have occurred together. In this case, soil pollutants may occur together due to sediment deposits from the flow of the river, during floods, ponctual spills, etc. Of course, we could use a collection of individual models, but when outputs are correlated, they inform each other.

In the upper knowledge requirements, I specified that you need a beginner level in Gaussian processes. Gaussian processes are congnitively intensive, but mathematically powerful. They are also, in my view, poetic. Basically, a Gaussian process  ($\mathcal{GP}$) is a multivariate-normal distribution with infinite dimensions, relying on continuous functions to define the mean vector and the covariance matrix. We can set the mean to zero when data are centered to 0. Covariance functions are named *kernels* , and many different kernels are distributed by *PyMC*. In single output Gaussian processes, we contruct a kernel on the inputs.

$$
y_i = f(x_i) + \epsilon_i
$$
$$
f(x_i) \sim \mathcal{GP}(0, k(x_i, x^\prime))
$$
In other words, we estimate $y$ as a function of $x$, and this function is a Gaussian process (infinite multivariate normal) with 0 mean and a covariance defined by a function between a pair of observations $x$ and $x \prime$.

In multi-output Gaussian processes, we add another kernel on the outputs. In *PyMC* (and *GPflow*), this approach is included with a strategy called *coregionalization* (formally, [intrinsic model of coregionalization](https://gpflow.readthedocs.io/en/master/notebooks/advanced/coregionalisation.html)). Hence, a pair of $\mathcal{GP}$ functions $f(x_i)$ and $f(x_j)$ will have a covariance equal to our kernel times a $B$ modifier matrix.

$$
cov \left( f_i(x), f_j(x \prime) \right) = k(x, x \prime) \cdot B
$$
Just like our kernel, this $B$ matrix must be positive-definite. It is convenient to define $B$ by two objects, $W$ and $\kappa$.

$$
B = WW^T + diag(\kappa)
$$
*PyMC* assembles the multioutput kernel for you, as long as you provide a kernel for the features, as well as $W$ and $\kappa$.

# Prepare the notebook

Nextjournal comes with Python and package installers *pip* and *conda*. We need to install *GeoPandas*, *Lets-plot* and *PyMC* - other packages are already install in Nextjournal's default Python environment. Because Lets-plot is not available in *conda*, for simplicity we install everything with *pip* in a Bash cell. *PyMC* is on the edge between *PyMC3* and *PyMC* version 4. I will update the notebook once *PyMC* version 4 becomes stable enough.

In [1]:
%%sh
pip install geopandas pymc3 lets-plot

The data from the Meuse river are distrubuted as shape files on the [website companion of the book ](http://spatial-analyst.net/book/meusegrids)*[A Practical Guide to Geostatistical Mapping](http://spatial-analyst.net/book/meusegrids)*, by Tomislav Hengl. I unzipped the file, opened it in [QGIS](https://www.qgis.org), then exported the data as a csv file.

[meuse.csv](https://nextjournal.com/data/QmZxY8abmYTJPbi9N47tmUPNYHcG88ggcisejTsFpRcaFT?content-type=text/csv&node-id=8bcf5ea5-7d49-4827-b8ef-98f0e63b5057&filename=meuse.csv&node-kind=file)

# Import packages and data

Packages are installed, but still not loaded in our Python session.

In [2]:
# Generic math
import numpy as np

# Tableaux
import pandas as pd # tabular data
import geopandas as gpd # spatial data

# Graphiques
import lets_plot as lp # interactive grammar of graphics
from lets_plot import tilesets # to define the map tiles
lp.LetsPlot.setup_html(isolated_frame=True) # so that lets-plot works in Nextjournal

# Modelling
import pymc3 as pm # bayesian modelling, including Gaussian processes

np.random.seed(629545) # random.org

WARNING (theano.link.c.cmodule): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


Similarly, data are loaded in Nextjournal, but not to our Python session.

In [3]:
meuse_df = pd.read_csv("meuse.csv")
meuse_df.head()

,x,y,cadmium,copper,lead,zinc,elev,dist,om,ffreq,soil,lime,landuse,dist.m
0,181072,333611,11.7,85,299,1022,7.909,0.001358,13.6,1,1,1,Ah,50
1,181025,333558,8.6,81,277,1141,6.983,0.012224,14.0,1,1,1,Ah,30
2,181165,333537,6.5,68,199,640,7.800,0.103029,13.0,1,1,1,Ah,150
3,181298,333484,2.6,81,116,257,7.655,0.190094,8.0,1,2,0,Ga,270
4,181307,333330,2.8,48,117,269,7.480,0.277090,8.7,1,2,0,Ah,380


As written on the website distributing the Meuse data set, coordinates `x` and `y` are in the `proj4: +init=epsg:28992` coordinate system. If we only care about modelling, we wouldn't need to mind much about the coordinate system. But since we care about plotting data on a map and later perform spatial queries, I specified in *GeoPandas* that `x` and `y` are point geometries expressed in the 28992 coordinate reference system (`crs`), then transformed the geometries to longitude-latitude angular data with the universal WGS84 system, i.e. 4326 (go to [espg.io](http://espg.io) to get the numbers). Moreover, later on, we will use the distance from the river as predictor. 

In [4]:
meuse_gdf = (
  gpd.GeoDataFrame(
      meuse_df,
      geometry = gpd.points_from_xy(meuse_df["x"], meuse_df["y"])
  )
  .set_crs(crs = 28992)
  .to_crs(crs = 4326)
)
meuse_gdf.head() # quick check

,x,y,cadmium,copper,lead,zinc,elev,dist,om,ffreq,soil,lime,landuse,dist.m,geometry
0,181072,333611,11.7,85,299,1022,7.909,0.001358,13.6,1,1,1,Ah,50,POINT (5.75854 50.99156)
1,181025,333558,8.6,81,277,1141,6.983,0.012224,14.0,1,1,1,Ah,30,POINT (5.75786 50.99109)
2,181165,333537,6.5,68,199,640,7.800,0.103029,13.0,1,1,1,Ah,150,POINT (5.75986 50.99089)
3,181298,333484,2.6,81,116,257,7.655,0.190094,8.0,1,2,0,Ga,270,POINT (5.76175 50.99041)
4,181307,333330,2.8,48,117,269,7.480,0.277090,8.7,1,2,0,Ah,380,POINT (5.76186 50.98903)


In [5]:
# updated

# quick check
(
    lp.ggplot() +
    lp.geom_point(data=meuse_gdf) +
    lp.theme_classic()
)

# Show geographic data on a map background

In the `meuse_gdf` geographic data frame, we have a column for spatial data named `geometry`. *Lets-plot* will not plot the geometry column, but ordinary columns. We have `x` and `y` in the table, but they are still expressed in EPSG 28992 (the geometry is in 4326, but the original `x` and `y` have not been modified). Let's update these values.

In [6]:
meuse_gdf['x'] = meuse_gdf["geometry"].x
meuse_gdf['y'] = meuse_gdf["geometry"].y

If you are familliar with *ggplot* (a widely known package in the R statistical programming language), you will understand the following code block right away. Basically, the grammar of graphics implemented in *ggplot* and *Lets-plot* consists in calling the data and stack layers of graphical entities, some mapping elements (*aesthetics* like x-y position, colors, point shape, etc.) to columns. In the following code block, I call the data, add a map, add points filled by a color according to values in the `copper` column, then customize the colors of the filling.

> *Lets-plot* proposes [a large number of map styles](https://lets-plot.org/pages/api.html#geospatial) (*tilesets*). I used the `STAMEN_DESIGN_TONER` tile set because it's undisruptive, has great contrasts, and is pleasant to my eyes.

In [7]:
# updated

(
    lp.ggplot() +
    lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) + # add map with specific tileset
    lp.geom_point(lp.aes(fill = "copper"), data = meuse_gdf,
                  size = 3, shape = 21, color = "black") +
    lp.scale_fill_brewer(type = "seq", palette = "Reds") # custom fill color gradients (red is pretty)
)

If you know R's *ggplot*, plotting for multiple metals would consist in, first, pivotting the table in long format, with all metal concentration values in a single column and the name of the metal variable in another column. Then, we would use facetting (`facet_wrap` or `facet_grid` to obtain multiple plots). But *Lets-plot* doesn't have [yet](https://github.com/JetBrains/lets-plot/issues/451) the option to plot a legend per facet, so zinc concentrations would take all fill color gradients, while cadmium point will all appear white since they are all low concentrations compared to zinc.

The trick for now (I will update the code when *Lets-plot* will implement some scaling argument) is to define the plot in a function and plot each metal one after the other with `lp.GGBunch()`, which is kind of tricky since it needs the position (below, `w` and `h`) of the plot from the top-left corner of the whole patchwork, but works great.

In [8]:
# updated

def plot_observations(metal):
    p = (
        lp.ggplot() +
        lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) +
        lp.scale_fill_brewer(type = 'seq', palette = 'Reds') +
        lp.geom_point(lp.aes(fill = metal), data = meuse_gdf,
                      size = 2, shape = 21, color = 'black')
    )
    return p

w, h = 480, 320
offset = 15
bunch = lp.GGBunch()
bunch.add_plot(plot_observations('cadmium'), 0, 0, w, h)
bunch.add_plot(plot_observations('copper'), w + offset, 0, w, h)
bunch.add_plot(plot_observations('lead'), 0, h + offset, w, h)
bunch.add_plot(plot_observations('zinc'), w + offset, h + offset, w, h)
bunch.show()

# Perform spatial queries

The `dist` column of the `meuse_gdf` table contains the information of the distance between the river and the sample. However, if we wish to evaluate the distance at any point in the domain of application of our model, in order to subsequently make our projections in space, we will have to be able to compute the distance to the river.

> Note that we are working here with geographic coordinates (longitude, latitude), which we approximate as geometric data. At this scale and for our usage, it is appropriate. But on a smaller scale, close to the poles, these distances could be distorted.

We will need the shape of the river to compute the distance to it at any point. I drew a line with mouse and click in QGIS along the center of the river. Since a line is difficult to handle with a csv, I exported it in the geojson format, which can be imported without ache with *GeoPandas*.

[river.geojson](https://nextjournal.com/data/QmYtVaAMkyfZyceU7WpsmK7sEfqK5SPshsLA6qUo16fgL6?content-type=application/geo%2Bjson&node-id=e5b4557b-3cb0-4192-b965-64562b43b278&filename=river.geojson&node-kind=file)


In [9]:
# updated

river = gpd.read_file("river.geojson")
# quick check
(
    lp.ggplot() +
    lp.geom_path(data=river) +
    lp.theme_classic()
)

I used the `distance` function of *GeoPandas* in a loop across rows, and assigned a new column named `distance_river`.

In [10]:
meuse_gdf = meuse_gdf.assign(
    distance_river = [meuse_gdf["geometry"][i].distance(river["geometry"][0]) for i in range(meuse_gdf.shape[0])]
)
meuse_gdf.head()

,x,y,cadmium,copper,lead,zinc,elev,dist,om,ffreq,soil,lime,landuse,dist.m,geometry,distance_river
0,5.758536,50.991562,11.7,85,299,1022,7.909,0.001358,13.6,1,1,1,Ah,50,POINT (5.75854 50.99156),0.000766
1,5.757863,50.991088,8.6,81,277,1141,6.983,0.012224,14.0,1,1,1,Ah,30,POINT (5.75786 50.99109),0.000840
2,5.759855,50.990893,6.5,68,199,640,7.800,0.103029,13.0,1,1,1,Ah,150,POINT (5.75986 50.99089),0.001930
3,5.761746,50.990410,2.6,81,116,257,7.655,0.190094,8.0,1,2,0,Ga,270,POINT (5.76175 50.99041),0.003276
4,5.761863,50.989026,2.8,48,117,269,7.480,0.277090,8.7,1,2,0,Ah,380,POINT (5.76186 50.98903),0.004521


Let's see what it looks like and how our computed distance compares to the `dist` column.

In [11]:
dist_compare = (
    lp.ggplot(meuse_gdf) + # call the data set
    lp.geom_point(lp.aes(x = "dist", y = "distance_river"), size = 3, color = "#ca0020") + # a layer of points
    lp.labs(x = "Distance in meuse.csv (km)", y = "Distance computed with GeoPandas (m)") # axes titles
)

In [12]:
# updated

dist_map = (
    lp.ggplot() +
    lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) + # add a layer of map tiles
    lp.geom_point(lp.aes(fill = "distance_river"), data = meuse_gdf, size = 3, shape = 21, color = "black") + # add a layer of points filled according to the distance
    lp.scale_fill_brewer(type = "seq", palette = "Reds") + # fill colors
    lp.labs(x = "Longitude", y = "Latitude") # axes titles
)

In [13]:
w, h = 480, 480
offset = 15
bunch = lp.GGBunch() # plot both
bunch.add_plot(dist_compare, 0, 0, w, h)
bunch.add_plot(dist_map, w + offset, 0, w, h)
bunch.show()

The left plot shows that distances are comparable, and the right plot shows that points away from the river are going darker. Good!

# Model spatial attributes with Gaussian processes

This part is probably the most difficult. Our workflow is the following.

1. Assign observations to training and testing
2. Preprocess data (standardize to 0 mean and 1 variance)
3. Arrange data to a format amenable to coregionalisation with *PyMC*
4. Create the model
5. Fit the model
6. Predict and criticize the model
7. Use the model

## Asssign to train / test

This is a necessary step to check if the model is good to predict at locations unknown by the model. This must be done with randomness, and it's usually good to back check if distributions in both sets are similar (I won't do it here though). I retain 70% of data in training, the rest in testing.

In [14]:
obs_id = meuse_gdf.index
n_train = np.round(obs_id.shape[0] * 0.7, 0).astype("int")
id_train = np.random.choice(obs_id, size = n_train, replace = False)
id_test = obs_id[~obs_id.isin(id_train)].values

I prefer to perform the split operation after preprocessing and arrangements.

## Preprocess data

I identify the columns I want to predict (targets, outcomes or outputs) and those containing the information needed to predict (features).

In [15]:
targets = ["cadmium", "copper", "lead", "zinc"]
features = ["x", "y", "distance_river"]

The data set contains other information useful to predict soil pollutants. I could have used the elevation, flood frequency, soil type, etc. But remember that if you want to predict pollutants at any location on your map, you also need to provide the features at any location. What would you do to obtain them? Think about it 30 seconds...

⏲️

For categorical variables (e.g. `soil`), you could draw polygons in your GIS to delimit zones and perform spatial queries to assign the category to enclosure in a polygon. But this would be impossible for numerical variables. In all cases (categories and numerical), you could create a simple spatial model, e.g. a *k*-nearest neighbors model with scikit-learn with the variable of interest (e.g. `elev`) as target and the position (`x` any `y`), then use this model to predict at any location what would become a feature for the soil pollutant model. By doing so, you should fit the soil pollutant model on the outcomes of your first spatial model, not the original points.

I won't do that here, since it would just make the approach more complicated - we will stick on the position and the distance from the river as features.

In [16]:
XY = meuse_gdf[targets + features]

The `XY` table must be standardized (or scaled, `_sc`) *on the training set*. Most tutorials online use scikit-learn tools, but since standardizing is a very simple operation, I prefer to stay explicit.

In [17]:
mean_sc = XY.loc[XY.index.isin(id_train), :].mean(axis = 0)
std_sc = XY.loc[XY.index.isin(id_train), :].std(axis = 0)
XY_sc = XY.apply(lambda x: (x-mean_sc)/std_sc, axis = 1)

> Note that concentration values are compositional data. Usually, I would have transformed them to log-ratios before scaling, but this would have complicated this workflow. For more info, read *[Why, and How, Should Geologists Use Compositional Data Analysis](https://en.wikibooks.org/wiki/Why,_and_How,_Should_Geologists_Use_Compositional_Data_Analysis)*[, by Ricardo A. Valls (2008)](https://en.wikibooks.org/wiki/Why,_and_How,_Should_Geologists_Use_Compositional_Data_Analysis).

## Arrange data

Coregionalization in *PyMC* needs a special format: the target is a single column of a matrix in long format, stacking all outputs into a vector and adding an integer (starting from 0) in the feature matrix indentifying to which output it belongs.

The first step to arrange our data set is to create a long format along the targets with the `melt()` method. The `variable` contains the target identifier and the `value` column contains its value. o keep track of the index for train/test split down the road, I "reset" the index so that it becomes a genuine column.

In [18]:
XY_m = (
  XY_sc
  .reset_index()
  .melt(id_vars = ["index"] + features, value_vars = targets)
)
XY_m.sample(8)

,index,x,y,distance_river,variable,value
501,36,0.791636,0.751319,-0.226376,zinc,0.249071
570,105,0.547928,-0.162460,2.308461,zinc,-0.965945
506,41,0.696871,0.514527,0.278263,zinc,-0.831540
82,82,-1.079929,-0.437228,-1.151299,cadmium,0.763428
525,60,-0.662126,0.277447,-1.159698,zinc,1.157645
353,43,0.497786,0.362794,0.326987,lead,-0.835414
76,76,-1.284631,-1.062300,0.169478,cadmium,-0.189754
561,96,-0.753365,-0.823206,0.927265,zinc,-0.672943


As I wrote, the variable must be an interger, while the variables in previous table are specified as strings. We can create a helper table, then merge it to our initial table.

In [19]:
variable_ids = pd.DataFrame(dict(
    variable = targets,
    variable_id = np.arange(0, len(targets))
))
variable_ids

,variable,variable_id
0,cadmium,0
1,copper,1
2,lead,2
3,zinc,3


In [20]:
XY_m = XY_m.merge(variable_ids, on = "variable")
XY_m.sample(8)

,index,x,y,distance_river,variable,value,variable_id
443,133,0.570346,0.045421,1.593152,lead,-0.953772,2
328,18,0.785088,1.129623,-1.136317,lead,0.484726,2
276,121,-0.509881,0.294803,-0.668510,copper,-0.642702,1
594,129,0.883691,1.162204,-0.883033,zinc,-0.006297,3
561,96,-0.753365,-0.823206,0.927265,zinc,-0.672943,3
66,66,-0.740633,-0.485432,-0.030870,cadmium,1.160587,0
396,86,-1.329603,-0.720567,-1.007330,lead,0.566666,2
610,145,-1.044388,-1.812143,-1.229843,zinc,0.364659,3


## Create the model

*PyMC* works with directed acyclic graphs, a fancy word referring to variables plugged together to generate an outcome (without feedback loops since they are "acyclic"). I consider the model to be something fixed, where I don't specify if it's for training or testing. This is why I use the `_mod` subscript instead of `_train` to define the objects in my model.

In [21]:
X_mod = XY_m.loc[XY_m["index"].isin(id_train), ["variable_id"] + features].values
id_col = 0
n_mod = n_train
y_mod = XY_m.loc[XY_m["index"].isin(id_train), "value"].values

In *PyMC*, models are defined in a `with` statement. At this point you might want to have a look a the model in the `with` statement below, then scroll back up. The block `## Feature covariance` in the model code block below defines the kernel of the features (x-y coordinates and distance to the river). *PyMC* allows to define priors on the hyperparameters of the kernel (lenght scale and amplitude). I used a half-Cauchy prior on length scale and another on amplitude. I tend to intensively use Cauchy priors since they have a thicker tail than normal distributions, thus allowing more extreme values. Half Cauchy, like half normal, restricts to positive values (and by default comes with zero mean). Such poorly informative priors allow a lot of flexibility. The following plot shows a half Cauchy prior where the value is probably close to zero, but might be higher.

In [22]:
# updated

from scipy import stats
x = np.linspace(0, 30, 100)
data = {
    'x': list(x) + list(x),
    'y': list(stats.halfcauchy.pdf(x, 0, 10)) + list(stats.halfnorm.pdf(x, 0, 10)),
    'stat': ["Half-Cauchy"] * x.size + ["Half-normal"] * x.size,
}
(
    lp.ggplot(data) +
    lp.geom_area(lp.aes(x = 'x', y = 'y', fill = 'stat'),
                 position = 'identity', color = 'white', alpha = .6, size = 0) +
    lp.scale_fill_manual(values = ["#ca0020", "#aaaaaa"]) +
    lp.labs(x = "x", y = "density")
)

In the kernel definition (a `Matern32` kernel), we must specify the input dimensions (`input_dim`), which is the dimension of the feature matrix, including the output index. The active dimensions (`active_dims`) are the column indexes of the features included in the kernel, i.e. excluding the output index. In our case, the output column index is `0`, so the feature column indexes are `np.arange(1, X_mod.shape[1])`.

The `## Coregion covariance` part also contain two hyperparameters I introduced in the Gaussian process introductory section above. The fist is `W`. It must be built as a `n_mod` by `n_mod` matrix. I chose to have independent normally distributed values at each element of the matrix. The other is `kappa`, always positive with a half Cauchy distribution and poorly informative. W and kappa could have been defined as fixed numbers, but distributions hold uncertainty and we Bayesians are never really sure of anything, right? ... Right?

We must specify a noise parameter to allow the Gaussian process have prediction errors, avoiding over-fitting. It's also specified as a half Cauchy.

The `pm.gp.Marginal()` function specifies that we need an exact Gaussian process, i.e. to predict continuous values. For categorical outputs, or even Poisson, we might select a `pm.gp.Latent()` function. The final resulting model is specified in the `gp.marginal_likelihood()` function.

In [23]:
with pm.Model() as model:
    # GP
    ## Feature covariance
    length_scale = pm.HalfCauchy("length_scale", beta = 10)
    amplitude = pm.HalfCauchy("amplitude", beta = 10)
    cov_feature = amplitude ** 2 * pm.gp.cov.Matern52(
        input_dim = X_mod.shape[1],
        ls = length_scale,
        active_dims = np.arange(1, X_mod.shape[1]) # all except index
    )

    ## Coregion covariance
    W = pm.Normal(
        "W", mu = 0, sd = 5,
        shape = (n_mod, n_mod)
    )
    kappa = pm.HalfCauchy("kappa", beta = 10, shape = n_mod)
    coreg = pm.gp.cov.Coregion(
        input_dim = X_mod.shape[1],
        active_dims = [id_col], # only index
        kappa = kappa,
        W = W
    )

    ## Combined covariance
    cov_f = coreg * cov_feature

    ## GP noise
    σ = pm.HalfCauchy("σ", beta = 10)

    ## Gaussian process
    gp = pm.gp.Marginal(cov_func = cov_f)

    ## Marginal likelihood
    y_ = gp.marginal_likelihood("y", X = X_mod, y = y_mod, noise = σ)


You can find the C code in this temporary file: C:\Temp\theano_compilation_error_qlr7gcp1


Exception: ("Compilation failed (return status=1): C:\\Temp\\ccq41m53.o: In function `run':\r. C:/Users/Artem Smirnov/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.7.12-64/tmpkyt46w3l/mod.cpp:85: undefined reference to `__imp__Py_NoneStruct'\r. C:/Users/Artem Smirnov/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.7.12-64/tmpkyt46w3l/mod.cpp:110: undefined reference to `__imp_PyExc_ValueError'\r. C:/Users/Artem Smirnov/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.7.12-64/tmpkyt46w3l/mod.cpp:116: undefined reference to `__imp_PyExc_RuntimeError'\r. C:/Users/Artem Smirnov/AppData/Local/Theano/compiledir_Windows-7-6.1.7601-SP1-Intel64_Family_6_Model_94_Stepping_3_GenuineIntel-3.7.12-64/tmpkyt46w3l/mod.cpp:144: undefined reference to `__imp_PyExc_NotImplementedError'\r. C:/Users/Artem Smirnov/AppData/Local/Thea

## Fit the model

There are two ways to fit this model. The `find_MAP()` function finds the *maximum a posteriori*. It's not a sampling algorithm. It just finds the optimum. It's quick, but it doesn't return the probabilistic hyperparameters and is at risk for finding local optima. On the other hand, a fully Bayesian approach with the `trace()` function can be very long to compute in multi-output mode.

In [ ]:
with model:
    # Fit
    # trace = pm.sample(2000, tune = 500, chains = 4, cores = 4, return_inferencedata = True) # long
    mp = pm.find_MAP() # quick

## Predict and criticize the model

The prediction is also done in a `with` statement. We take 50 samples of the outcome. Usually, I would have taken more, but let's keep the memory low here.

In [ ]:
 with model:
    pred_ = gp.conditional("pred_", XY_m[["variable_id"] + features].values)
    pred_samples = pm.sample_posterior_predictive([mp], var_names=["pred_"], samples = 50)

I compute the mean of the 50 samples, then tag if the observation is in the training or testing set.

In [ ]:
XY_m["yhat"] = pred_samples["pred_"].mean(axis = 0)
XY_m["tr_te"] = "train"
XY_m.loc[XY_m["index"].isin(id_test), "tr_te"] = "test"
XY_m.sample(8)

It would be nice to have an idea of the prediction error. But if I compute the error between `value` and `yhat`, the error will also be scaled and difficult to interpret. So I add columns in the original scale.

In [ ]:
XY_m["value_original"] = 0
XY_m["yhat_original"] = 0
for metal in targets:
    XY_m.loc[XY_m["variable"] == metal, "value_original"] = XY_m.loc[XY_m["variable"] == metal, "value"] * std_sc[metal] + mean_sc[metal]
    XY_m.loc[XY_m["variable"] == metal, "yhat_original"] = XY_m.loc[XY_m["variable"] == metal, "yhat"] * std_sc[metal] + mean_sc[metal]

This long-format table is ready to be used with *Pandas* for statistics the grammar of graphics of *Lets-plot*. First, the root-mean-square-error.

In [ ]:
rmse = (
    XY_m
    .query("tr_te == 'test'")
    .groupby('variable')
    .apply(func = lambda x: np.mean((x.value_original - x.yhat_original)**2)**0.5)
    .round(2)
)
rmse

The acceptability of these errors depend on the usage of the model.

In [ ]:
(
    lp.ggplot(XY_m, lp.aes(x = "value", y = "yhat")) +
    lp.facet_grid(x = "tr_te", y = "variable") +
    lp.geom_abline(intercept = 0, slope = 1, color = 'red', size = 1) +
    lp.geom_point()
)

The testing prediction is not so good, with some off points. The model could be improved by adding more predictors and adjusting hyper-parameters (which should be optimized by `find_MAP()`, though). Coregionalization may not be needed and simpler models would perform better. In fact, a much simpler approach with generalized additive models with *PyGAM* returned similar results with the same features. If you want to see how I did it with GAMs, go [here - (](https://gitlab.com/essicolo/blog_sm/-/blob/master/2022-03-14_spatial/2022-03-14_gam.ipynb)the following code exports data for GAMs).

In [ ]:
import pickle
with open("results/to_gam.pkl", "wb") as file:
    pickle.dump([XY, XY_sc, id_train, id_test, mean_sc, std_sc, targets, features], file)

## Use the model

Once we have our model, we can project it onto our map. To do this, one can use a grid of points, and evaluate the metal contents on this grid. There are several ways to create grids. The *Pandas* package, although not offering a function to create grids of points, does offer the `expand_grid` function [in its documentation](https://<https://pandas.pydata.org/docs/user_guide/cookbook.html> ?highlight=cookbook#creating-example-data).

In [ ]:
import itertools
def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())

In [ ]:

gridres = 0.0012 # grid resolution : could be finer
x_grid = np.arange(5.71, 5.77, gridres)
y_grid = np.arange(50.95, 51.0, gridres)

squaregrid = expand_grid(
    {"x": x_grid, "y": y_grid}
)

(
    lp.ggplot(squaregrid, lp.aes('x', 'y')) +
    lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) +
    lp.geom_point()
)

The problem with this grid is that it goes far away from our domain. Indeed, predicting concentrations on the West side of the river is risky, since we only have data between the East side and the canal. I drew a polygon with QGIS, exported it in the geojson format, then performed a spatial query to filter out points outside the polygon. The grid must first be turned to a *GeoPandas* data frame.

In [ ]:
geogrid = gpd.GeoDataFrame(
    squaregrid,
    geometry = gpd.points_from_xy(squaregrid["x"], squaregrid["y"], crs = "epsg:4326")
)

[polygon.geojson](https://nextjournal.com/data/QmXWCH5fTcmVwCRFViWs8pkCfCmEmqqAzWwU4cvyXZzMw2?content-type=application/geo%2Bjson&node-id=df5d18eb-d37a-4ed0-8591-9920b437fed1&filename=polygon.geojson&node-kind=file)


In [ ]:
geo_polygon = gpd.read_file("/.nextjournal/data-named/QmXWCH5fTcmVwCRFViWs8pkCfCmEmqqAzWwU4cvyXZzMw2/polygon.geojson")
grid_filter = np.array([geogrid["geometry"][i].within(geo_polygon["geometry"][0]) for i in range(geogrid.shape[0])])

(
  lp.ggplot(geogrid.iloc[grid_filter, :]) +
  lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) +
  lp.geom_point(lp.aes(x = 'x', y = 'y'))
)

Since the grid is good, I save it in its own object.

In [ ]:
geogrid_f = geogrid.iloc[grid_filter, :]

The resolution of the grid could be finer, but it is better for the example to limit it to little. Let us now calculate the distances from the river for each of the points in the grid.

In [ ]:
geogrid_f = geogrid_f.assign(
   distance_river = [geogrid_f["geometry"][i].distance(river["geometry"][0]) for i in geogrid_f.index]
)

(
    lp.ggplot(geogrid_f) +
    lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) +
    lp.geom_tile(lp.aes('x', 'y', fill = 'distance_river'), alpha = 0.8) +
    lp.scale_fill_brewer(type = 'seq', palette = 'Reds')
)

To obtain the desirable prediction, we have to format the data just like the data we fitted the model onto. Remember that the features were scaled, and that the first column contained the index of the target.

In [ ]:
geogrid_sc = (geogrid_f[features] - mean_sc[features]) / std_sc[features]

In [ ]:
geogrid_mod = pd.concat([geogrid_sc, geogrid_sc, geogrid_sc, geogrid_sc]).reset_index(drop = True)
geogrid_varid = pd.Series(np.repeat([0, 1, 2, 3], geogrid_sc.shape[0]), name = "variable_id")
geogrid_mod = pd.concat([geogrid_varid, geogrid_mod], axis = 1)

In [ ]:
with model:
    predgrid = gp.conditional("predgrid", geogrid_mod.values)
    predgrid_samples = pm.sample_posterior_predictive([mp], var_names=["predgrid"], samples = 50)

I compute the mean of the 50 samples, then tag if the observation is in the training or testing set.

In [ ]:
geogrid_mod["yhat"] = predgrid_samples["predgrid"].mean(axis = 0)
geogrid_mod = geogrid_mod.merge(variable_ids, on = "variable_id")

It would be nice to have an idea of the prediction error. But if I compute the error between `value` and `yhat`, the error will also be scaled and difficult to interpret. I add columns in the original scale.

In [ ]:
for var in targets:
    geogrid_mod.loc[geogrid_mod["variable"] == var, "yhat"] = geogrid_mod.loc[geogrid_mod["variable"] == var, "yhat"] * std_sc[var] + mean_sc[var]

In [ ]:
for var in features:
    geogrid_mod.loc[:, var] = geogrid_mod.loc[:, var] * std_sc[var] + mean_sc[var]

In [ ]:
geogrid_mod

We can plot the spatial distribution on the prediction as we did before.

In [ ]:
def plot_predictions(metal):
    p = (
            lp.ggplot() +
            lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) +
            lp.scale_fill_brewer(type = 'seq', palette = 'Reds') +
            lp.geom_tile(data = geogrid_mod.loc[geogrid_mod["variable"] == metal, :], mapping = lp.aes('x', 'y', fill = 'yhat'), alpha = 0.85) +

            lp.geom_point(data = meuse_gdf, mapping = lp.aes(x = 'x', y = 'y', fill = metal), size = 2, shape = 21, color = 'black')
    )
    return p

w, h = 480, 320
offset = 15
bunch = lp.GGBunch()
bunch.add_plot(plot_predictions('cadmium'), 0, 0, w, h)
bunch.add_plot(plot_predictions('copper'), w + offset, 0, w, h)
bunch.add_plot(plot_predictions('lead'), 0, h + offset, w, h)
bunch.add_plot(plot_predictions('zinc'), w + offset, h + offset, w, h)
bunch

When I ran `predgrid_samples = pm.sample_posterior_predictive([mp], var_names=["predgrid"], samples = 50)`, I drew 50 samples from a conditionned Gaussian process distribution distribution. Then I computed the mean of these 50 samples to obtain a prediction for each point on the grid. What could I also do with these samples?

⏲️

That's right, computing uncertainties!  Where there are fewer observations, Gaussian processes return larger uncertainties. Let us check it out.

In [ ]:
geogrid_mod["yhat_sd"] = predgrid_samples["predgrid"].std(axis = 0)
for var in targets:
    geogrid_mod.loc[geogrid_mod["variable"] == var, "yhat_sd"] = geogrid_mod.loc[geogrid_mod["variable"] == var, "yhat_sd"] * std_sc[var]

In [ ]:
def plot_predictions(metal):
    p = (
            lp.ggplot() +
            lp.geom_livemap(tiles = tilesets.STAMEN_DESIGN_TONER) +
            lp.scale_fill_brewer(type = 'seq', palette = 'Reds') +
            lp.geom_tile(data = geogrid_mod.loc[geogrid_mod["variable"] == metal, :], mapping = lp.aes('x', 'y', fill = 'yhat_sd'), alpha = 0.85) +

            lp.geom_point(data = meuse_gdf, mapping = lp.aes(x = 'x', y = 'y'), size = 2, shape = 1, color = 'black')
    )
    return p

w, h = 480, 320
offset = 15
bunch = lp.GGBunch()
bunch.add_plot(plot_predictions('cadmium'), 0, 0, w, h)
bunch.add_plot(plot_predictions('copper'), w + offset, 0, w, h)
bunch.add_plot(plot_predictions('lead'), 0, h + offset, w, h)
bunch.add_plot(plot_predictions('zinc'), w + offset, h + offset, w, h)
bunch

# Recapitulation

In this tutorial, we began by the import of a tabular *csv* in a *Pandas* data frame. We transformed this table to a *GeoPandas* data frame, i.e. data frame including a geometry column. By transforming the projection of our points to the correct coordinate system, we could project our points on a map with *Lets-plot*.

We then imported a *geojson* file directly as a *GeoPandas* data frame in the aim of computing the distance to the river from any location. This was our first spatial query with *GeoPandas*.

We created a spatial model as conventionnally done with machine learning, with training and testing sets and preprocess. We used multioutput Gaussian processes with *PyMC*. We then sampled from our fitted Gaussian process to obtain our predictions and criticize our model.

We finally used our model to spatially project soil pollutants concentrations and their uncertainty on a grid contained in a polygon to avoid spatial extrapolations.

[Comment on Twitter](https://twitter.com/essicolo/status/1504537755916906500).

[Comment on Mastodon](https://mastodon.online/web/@essi/107977832169912555).